In [2]:
from utils import *
import getpass
# from landsatxplore.earthexplorer import EarthExplorer
import tarfile
import boto3

keep_original_band_scenes = False
one_per_month = True
dir_suffix = ""
if (dir_suffix != "") and (not dir_suffix.endswith("/")):
    dir_suffix = dir_suffix + "/"

In [8]:
# aois = kml_to_poly("data/inputs/aois.kml")
# list(aois.geoms)

In [ ]:
aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)
# aws_session = rasterio.session.AWSSession(boto3.Session())

In [ ]:
bbox = read_kml_polygon("data/inputs/LANDSAT_8_127111/WA.kml")[1]
bbox = resize_bbox(bbox, 0.1)

In [ ]:
user = getpass.getpass("Enter your username?")
password = getpass.getpass("Enter your password?")

In [ ]:
start_date = "2014-01-01T00:00:00"
end_date: str = "2016-01-01T00:00:00"
platform = "LANDSAT_7"
# cloud_cover = 80
# query = get_landsat_search_query(
#     [67.45, -72.55, 67.55, -72.45],
#     start_date=start_date,
#     end_date=end_date,
#     platform=platform,
#     # cloud_cover=cloud_cover,
# )  # Amery ice shelf bed rock

# query = get_landsat_search_query(
#     [69.2, -68.1, 69.4, -67.9],
#     start_date=start_date,
#     end_date=end_date,
#     platform=platform,
#     cloud_cover=50,
# )  # Top of Amery ice shelf

# query = get_landsat_search_query(
#     bbox,
#     start_date=start_date,
#     end_date=end_date,
#     platform=platform,
# )  # WA sand dunes

# Landsat 7 Amery iceshelf
query = get_search_query(
    [67.45, -72.55, 67.55, -72.45],
    start_date="",
    platform=platform,
)  # Amery ice shelf bed rock

server_url = "https://landsatlook.usgs.gov/stac-server/search"
features = query_stac_server(query, server_url)
# scene_dict, scene_list = find_landsat_scenes_dict(features, one_per_month)
scene_dict, scene_list = find_scenes_dict(
    features, one_per_month, start_end_years=[1999, 2002]
)

In [ ]:
len(scene_list)

In [ ]:
path_rows = list(scene_dict.keys())
path_rows

In [ ]:
dates = [list(scene_dict[pr].keys()) for pr in path_rows]
dates

In [ ]:
path_row = "127111"
path_row_idx = path_rows.index(path_row)
aws_reds = [scene_dict[path_row][k][0]["red"][1] for k in scene_dict[path_row]]
bounds_dict = {}
for r in aws_reds:
    id = os.path.basename(r).split("_")[2]
    meta = stream_scene_from_aws(r, aws_session, metadata_only=True)[1]
    bounds_dict[id] = {"bounds": meta["bounds"], "crs": meta["crs"].data}

In [ ]:
for key in bounds_dict:
    bounds = bounds_dict[key]["bounds"]
    crs = bounds_dict[key]["crs"]
    bottom_utm = UTM(bounds.left, bounds.bottom)
    top_utm = UTM(bounds.right, bounds.top)
    bottom_lla = UTMtoLLA(bottom_utm, crs)
    top_lla = UTMtoLLA(top_utm, crs)
    print(key, bottom_lla, top_lla)

In [ ]:
date = dates[path_row_idx][0]

r_url = scene_dict[path_row][date][0]["red"][0]
g_url = scene_dict[path_row][date][0]["green"][0]
b_url = scene_dict[path_row][date][0]["blue"][0]

r_aws = scene_dict[path_row][date][0]["red"][1]
g_aws = scene_dict[path_row][date][0]["green"][1]
b_aws = scene_dict[path_row][date][0]["blue"][1]

In [ ]:
r_img, _ = stream_scene_from_aws(r_aws, aws_session)
g_img, _ = stream_scene_from_aws(g_aws, aws_session)
b_img, _ = stream_scene_from_aws(b_aws, aws_session)

In [ ]:
rgb = apply_gamma(cv.merge([r_img[0, :, :], g_img[0, :, :], b_img[0, :, :]]), 1.0, True)
plt.imshow(rgb)

In [ ]:
r_band_suffix = os.path.splitext(os.path.basename(r_url))[0].split("_")[-1]
g_band_suffix = os.path.splitext(os.path.basename(g_url))[0].split("_")[-1]
b_band_suffix = os.path.splitext(os.path.basename(b_url))[0].split("_")[-1]

In [ ]:
counter = 1
for pr in path_rows:
    true_color_dir = f"data/inputs/{dir_suffix}{platform}_{pr}/true_color"
    os.makedirs(true_color_dir, exist_ok=True)

    true_color_ds_dir = f"data/inputs/{dir_suffix}{platform}_{pr}/true_color_ds"
    os.makedirs(true_color_ds_dir, exist_ok=True)

    pr_dict = scene_dict[pr]
    for date in pr_dict:
        pr_date_list = pr_dict[date]
        for el in pr_date_list:
            print(
                f"Now downloading and processing {el['scene_name']} for date: {date} and path_row: {pr}, scene {counter} from total of {len(scene_list)}",
                end="\r",
            )
            counter += 1
            r_url = el["red"][1]
            g_url = el["green"][1]
            b_url = el["blue"][1]
            output_dir = (
                f"data/inputs/{dir_suffix}{platform}_{pr}/Originals/{el['scene_name']}"
            )
            os.makedirs(output_dir, exist_ok=True)
            r_output = os.path.join(output_dir, os.path.basename(r_url))
            g_output = os.path.join(output_dir, os.path.basename(g_url))
            b_output = os.path.join(output_dir, os.path.basename(b_url))
            r_img, r_meta = stream_scene_from_aws(r_url, aws_session)
            g_img, g_meta = stream_scene_from_aws(g_url, aws_session)
            b_img, b_meta = stream_scene_from_aws(b_url, aws_session)

            imgs = [r_img, g_img, b_img]
            outputs = [r_output, g_output, b_output]
            metas = [r_meta, g_meta, b_meta]
            for i, img in enumerate(imgs):
                with rasterio.open(outputs[i], "w", **metas[i]["profile"]) as ds:
                    ds.write(img[0, :, :], 1)

            files = glob.glob(f"{output_dir}/**")
            r_band = list(filter(lambda f: f.endswith(f"_{r_band_suffix}.TIF"), files))[
                0
            ]
            g_band = list(filter(lambda f: f.endswith(f"_{g_band_suffix}.TIF"), files))[
                0
            ]
            b_band = list(filter(lambda f: f.endswith(f"_{b_band_suffix}.TIF"), files))[
                0
            ]
            true_bands = [r_band, g_band, b_band]
            tc_file = (
                f"{os.path.join(true_color_dir, os.path.basename(output_dir))}_TC.TIF"
            )
            make_landsat_true_color_scene(true_bands, tc_file)
            downsample_dataset(
                tc_file, 0.2, os.path.join(true_color_ds_dir, os.path.basename(tc_file))
            )

            if not keep_original_band_scenes:
                shutil.rmtree(
                    f"data/inputs/{dir_suffix}{platform}_{pr}/Originals",
                    ignore_errors=True,
                )

In [ ]:
# ee = EarthExplorer(user, password)
# for scene in required_scenes:
#     output_dir = f"data/inputs/{platform}_{path_num}"
#     os.makedirs(f"data/inputs/{platform}_{path_num}", exist_ok=True)
#     print(f"Downloading {scene}")
#     ee.download(scene, output_dir)
# ee.logout()

In [ ]:
# tar_files = [
#     f for f in glob.glob(f"data/inputs/{platform}_{path_num}/**") if f.endswith(".tar")
# ]
# for tar_file in tar_files:
#     extraction_dir = os.path.join(
#         output_dir, os.path.splitext(os.path.basename(tar_file))[0]
#     )
#     os.makedirs(extraction_dir, exist_ok=True)
#     with tarfile.open(tar_file) as tf:
#         tf.extractall(extraction_dir)